In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import linregress
from scipy.stats import gaussian_kde
from scipy.optimize import curve_fit
from collections import defaultdict
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy import stats

In [ ]:
min_n_slices = 2
max_GFP_med = 10**2.5
min_GFP_tot = 100
min_frac_in_cell = 0.99

coefs = pd.read_csv('20230914_csvs/GFP_coefs.csv',names=[1,2]).values[0,:]
coefs

cmap = matplotlib.cm.get_cmap('tab10')
colors = [[0,0.85,0.85],'red']

cmap2 = matplotlib.cm.get_cmap('viridis')

replicates = ['20230914', '20240201']

In [ ]:
def get_smoothed_slope(data, ch, frac=0.3):
    
    x_fit = np.logspace(np.log10(np.min(data['N_MHC'])),np.log10(np.max(data['N_MHC'])),1000)
    
    y_smooth = lowess(exog=data['N_MHC'],endog=data[f'{ch}_tot'],
                      xvals=data['N_MHC'],return_sorted=False,frac=frac)
    fit_real = linregress(data['N_MHC'],y_smooth)
    
    y_smooth = lowess(exog=data['N_MHC'],endog=data[f'rand_{ch}_tot'],
                      xvals=data['N_MHC'],return_sorted=False,frac=frac)
    fit_rand = linregress(data['N_MHC'],y_smooth)
    
    return fit_real, fit_rand

def get_slope(data, ch, frac=0.3):
        
    fit_real = linregress(data['N_MHC'],data[f'{ch}_tot'])
    
    data_rand = data[~np.isnan(data[f'rand_{ch}_tot'])]
    fit_rand = linregress(data_rand['N_MHC'],data_rand[f'rand_{ch}_tot'])
    
    return fit_real, fit_rand

def filter_and_subtract(data, smooth=0.1, MHC_factor=1,
          min_n_slices=min_n_slices, max_GFP_med=max_GFP_med,
          min_GFP_tot=min_GFP_tot, min_frac_in_cell=min_frac_in_cell, coefs=coefs):
    
    data['N_GFP'] = ((data['GFP_tot'] - coefs[1])/coefs[0])
    data['N_MHC'] = ((data['GFP_tot'] - coefs[1])/coefs[0])*MHC_factor
    
    good_idx = (data['GFP_med'] < max_GFP_med) & \
           (data['n_slices'] >= min_n_slices) & \
           (data['frac_in_cell'] > min_frac_in_cell) & \
           (data['N_GFP'] > min_GFP_tot)

    filtered = data[good_idx]
    filtered['rep'] = data['fname'].map(lambda x: x[0])
    
    for i,ch in enumerate(['pCD3z','pLAT']):

        filtered[f'{ch}_tot_raw'] = filtered[f'{ch}_tot'] + filtered['volume']*filtered[f'{ch}_bkgnd']
        filtered[f'rand_{ch}_tot_raw'] = filtered[f'rand_{ch}_tot'] + filtered['volume']*filtered[f'rand_{ch}_bkgnd']

        x_base = np.log10(filtered['N_MHC'])
        baseline = lowess(exog=np.log10(filtered['N_MHC']),endog=np.log10(filtered[f'rand_{ch}_tot_raw']),
                          xvals=x_base,return_sorted=False,frac=smooth)
        baseline = 10**baseline

        filtered[f'{ch}_baseline'] = baseline
        filtered[f'{ch}_tot_sub'] = filtered[f'{ch}_tot_raw'] - filtered[f'{ch}_baseline']
        filtered[f'rand_{ch}_tot_sub'] = filtered[f'rand_{ch}_tot_raw'] - filtered[f'{ch}_baseline']
    
    return filtered

def filter_only(data, MHC_factor=1,
                min_n_slices=min_n_slices, max_GFP_med=max_GFP_med,
                min_GFP_tot=min_GFP_tot, min_frac_in_cell=min_frac_in_cell, coefs=coefs):
    
    data['N_GFP'] = ((data['GFP_tot'] - coefs[1])/coefs[0])
    data['N_MHC'] = ((data['GFP_tot'] - coefs[1])/coefs[0])*MHC_factor
    
    good_idx = (data['GFP_med'] < max_GFP_med) & \
           (data['n_slices'] >= min_n_slices) & \
           (data['frac_in_cell'] > min_frac_in_cell) & \
           (data['N_GFP'] > min_GFP_tot)

    filtered = data[good_idx].copy()
    filtered['rep'] = data['fname'].map(lambda x: x[0])
    
    return filtered

In [ ]:
fit_params = defaultdict(lambda: np.zeros([2,7]))
MHC_densities = []

for rep in replicates:
    for i,r in enumerate(['1-3','1-6','1-12','1-24','1-48','1-96','1-192']):

        data = pd.read_csv(f'{rep}_csvs/r{r}_5min.csv')

        ratio_split = [int(x) for x in r.split('-')]
        MHC_factor = ratio_split[0]/ratio_split[1]
        
        if rep == '20230914':
            MHC_densities.append(2400*3*MHC_factor)

        filtered = filter_only(data, MHC_factor=MHC_factor)

        for ch in ['pCD3z','pLAT']:
            fit_real, fit_rand = get_smoothed_slope(filtered,ch,frac=0.1)

            fit_params[f'{rep}_{ch}_real'][0,i] = fit_real.intercept
            fit_params[f'{rep}_{ch}_real'][1,i] = fit_real.slope
            fit_params[f'{rep}_{ch}_rand'][0,i] = fit_rand.intercept
            fit_params[f'{rep}_{ch}_rand'][1,i] = fit_rand.slope

In [ ]:
fit_params

In [ ]:
plot_info = {'real':['Arrays','-'],
             'rand':['Bkgnd','--'],
             '20230914': ['rep1','o',1],
             '20240201': ['rep2','o',0.7],
             'pLAT':np.array([1,0,0]),
             'pCD3z':np.array([0,0.85,0.85])
            }

In [ ]:
summary_data = pd.DataFrame()
summary_data['pMHC density'] = MHC_densities

for i,ch in enumerate(['pCD3z','pLAT']):
    for kind in ['real','rand']:
        pooled_vals = []
        for r,rep in enumerate(replicates):

            vals = fit_params[f'{rep}_{ch}_{kind}'][1,:]

            if kind == 'real':
                norm = fit_params[f'{rep}_{ch}_real'][1,0]

            vals = vals/norm

            summary_data[f'{ch}_{kind}_rep{r+1}'] = vals
summary_data

In [ ]:
summary_data.to_csv('array_density_range_activation_data.csv')

In [ ]:
def gauss_peak(x,h,s,dx,m):
    return h*np.exp(-((x-dx)**2)/(2*s**2)) + m

def flat_fit(x,m):
    return np.ones(x.shape)*m

plot_info = {'real':['Arrays','-'],
             'rand':['Bkgnd','--'],
             1: ['rep1','o',1],
             2: ['rep2','o',0.7],
             'pLAT':np.array([1,0,0]),
             'pCD3z':np.array([0,0.85,0.85])
            }

In [ ]:
data = pd.read_csv('array_density_range_activation_data.csv',index_col=0)
data

In [ ]:
MHC_densities = data['pMHC density'].tolist()

peaks = []
for i,ch in enumerate(['pCD3z','pLAT']):
    plt.figure(figsize=[9,6])
    for ii,kind in enumerate(['rand','real']):

        # plt.subplot(2,2,(2*i)+ii+1)
        
        pooled_vals = []
        for r in [1,2]:

            vals = data[f'{ch}_{kind}_rep{r}'].tolist()
            pooled_vals.append(vals)
            
            if kind == 'rand':
                color = np.array([0.7]*3)*plot_info[r][2]
            else:
                color=plot_info[ch]*plot_info[r][2]

            if kind == 'real':
                plt.plot(MHC_densities, vals, 'o', linewidth=4, markersize=10, color=color, label=f'{plot_info[r][0]}')
            else:
                plt.plot(MHC_densities, vals, 'o', linewidth=4, markersize=10, color=color, label=' ')

        if kind == 'rand':
            color = np.array([0.7]*3)
        else:
            color=plot_info[ch]

        if kind == 'real':
            for d in range(len(MHC_densities)):
                low = min(pooled_vals[0][d],pooled_vals[1][d])
                high = max(pooled_vals[0][d],pooled_vals[1][d]) - low
                plt.bar(MHC_densities[d]*1.005,high,bottom=low,width=MHC_densities[d]/10,alpha=0.3,color=color)

            plt.scatter(MHC_densities,np.mean(np.stack(pooled_vals,axis=1),axis=1),marker='_',color=color,s=200,linewidth=3,label='mean')
        else:
            plt.scatter(MHC_densities,np.mean(np.stack(pooled_vals,axis=1),axis=1),marker='',color=color,s=200,linewidth=3,label=' ')
        
        pooled_vals_array = np.concatenate(pooled_vals,axis=0)
        params_gauss,cov_gauss = curve_fit(gauss_peak,np.log10(MHC_densities*2),pooled_vals_array,
                                           p0=[np.max(pooled_vals_array),
                                               0.5,
                                               np.log10(MHC_densities*2)[pooled_vals_array == np.max(pooled_vals_array)][0],
                                               1],
                                           bounds=[[0,      np.log10(2)/2, 1, -3],
                                                   [np.inf, np.inf, 4,     np.inf]],
                                           maxfev=50000)

        params_lin,_ = curve_fit(flat_fit,np.log10(MHC_densities*2),pooled_vals_array,p0=[1],
                                 bounds=[[-np.inf],
                                         [np.inf]],
                                 maxfev=50000)

        err_gauss = np.sum((gauss_peak(np.log10(MHC_densities*2),*params_gauss) - pooled_vals_array)**2)
        err_lin = np.sum((flat_fit(np.log10(MHC_densities*2),*params_lin) - pooled_vals_array)**2)

        df_gauss = len(pooled_vals_array) - 4
        df_lin = len(pooled_vals_array) - 1
        
        F = ((err_lin - err_gauss)/(df_lin - df_gauss))/(err_gauss/df_gauss)
        p_val = 1 - stats.f.cdf(F,df_lin,df_gauss)

        x_fit = np.linspace(np.log10(30),np.log10(3000),1000)
        y_fit_gauss = gauss_peak(x_fit,*params_gauss)
        y_fit_lin = flat_fit(x_fit,*params_lin)

        if p_val < 0.05:
            gauss_color = 'black'
            lin_color = 'gray'
        else:
            gauss_color = 'gray'
            lin_color = 'black'
        if kind == 'real':
            plt.plot(10**x_fit,y_fit_gauss,linewidth=2,linestyle=plot_info[kind][1],color='k',label=f'fit')
        else:
            plt.plot(10**x_fit,y_fit_gauss,linewidth=2,linestyle=plot_info[kind][1],color='k',label=' ')
        # plt.plot(10**x_fit,y_fit_lin,'--',linewidth=2,color=lin_color,label='Flat fit')
            
        peaks.append([r, kind, ch, params_gauss, cov_gauss, params_lin, F, p_val, np.max(pooled_vals), np.min(pooled_vals)])
            
        plt.xscale('log')
        plt.xticks(fontsize=24)
        plt.yticks(fontsize=24)
    
#         if ch == 'pCD3z':
#                 plt.xticks([])
            
        plt.ylim([-1.8,7.7])
        
        legend = plt.legend(fontsize=20,title='Bkgnd   Arrays           ',ncol=2,columnspacing=1,labelspacing=0.1)
        legend.get_title().set_fontsize('20')
    plt.savefig(f'{ch}_desnsity_dependence_2rep_with_fits.png',dpi=1000,bbox_inches='tight')
    
peaks = pd.DataFrame(peaks, columns=['rep','kind','ch','params_gauss','cov_gauss','params_lin','F-val','p-val','max','min'])

In [ ]:
plt.figure(figsize=[6,6])
plt.plot([0,1],[0,1],'k-',label='Array fit',linewidth=3)
plt.plot([0,1],[0,1],'k--',label='Bkgnd fit',linewidth=3)
plt.legend(loc=[1.1,0.7],fontsize=20)
plt.savefig('fit_legend.png',dpi=1000,bbox_inches='tight')

In [ ]:
plt.figure(figsize=[6,6])
plt.plot([0,1],[0,1],'o',label='rep1',linewidth=3,color=np.array([0.7]*3)*plot_info[1][2],markersize=10)
plt.plot([0,1],[0,1],'o',label='rep2',linewidth=3,color=np.array([0.7]*3)*plot_info[2][2],markersize=10)
legend = plt.legend(loc=[1.1,0.7],fontsize=20,title='Bkgnd')
legend.get_title().set_fontsize('20')
plt.savefig('bkgnd_legend.png',dpi=1000,bbox_inches='tight')

In [ ]:
peaks['params_gauss'] = peaks['params_gauss'].map(lambda x: [round(val,3) for val in x])
peaks['max_val'] = peaks['params_gauss'].map(lambda x: x[0]+x[3])
peaks

In [ ]:
plt.figure(figsize=[3.5,6])

for i,ch in enumerate(['pCD3z','pLAT']):
    # plt.subplot(2,1,i+1)
    for ii,kind in enumerate(['real','rand']):
        row = peaks.loc[(peaks['kind']==kind)&(peaks['ch']==ch),:]
        err = np.sqrt(row['cov_gauss'].values[0][0,0] + row['cov_gauss'].values[0][3,3] + 2*row['cov_gauss'].values[0][0,3])
        height = row['max_val'].values[0]

        p_val = stats.norm.cdf(1,loc=height,scale=err)
        print(ch, kind, err, p_val)

        if kind == 'rand':
            color = np.array([0.7]*3)
        else:
            color=plot_info[ch]

        plt.bar(2.5*i + ii,height,yerr=err,color=color,capsize=10,error_kw={'capthick':3,'elinewidth':3})

xlim = plt.xlim()
plt.plot(xlim,[1]*2,'k--',linewidth=3)
plt.xlim(xlim)
plt.ylim([0,7.5])
plt.yticks(fontsize=24)

plt.xticks([0,1,2.5,3.5],['Arrays','Bkgnd']*2,fontsize=24,rotation=90)
    
plt.tight_layout()
plt.savefig('desnsity_dependence_max_height.png',dpi=1000,bbox_inches='tight')